TO DO (NLP HACK 2)

- make everything lower case
- try to pull out blood results

IDEAS/ THINGS TO TRY (NLP hack 1)
- clean away transaction records
- look for renal food purchase
- look in 'Assessment' for diagnosis
- find lab values using regex and add to column
- what do the most common lab patterns look like?
- have a look at everything with renal/kidney (e.g. we don't want acute kidney disease)
- look for CKD meds e.g. Semintra in Item Label, Item Name, Examination Text

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
columns = ['Database Name', 'Patient Number', 'Client Number', 'Consultation Number', 'Item Name', 'Units',
           'Item Label', 'Clinic Code', 'Consulting Vet', 'Patient Desexed', 'Deceased Date',
           'Visit Date', 'Consultation Date', 'Examination Text' ,'Patient Species', 'Patient Breed',
           'Patient Colour', 'Patient Sex', 'Patient Date of Birth', 'Insured', 'weight', 'temperature',
           'HeartRate', 'BodyScore', 'DentalGrade', 'RespRate', 'PainScore', 'BP', 'CRT', 'MMColour']

In [ ]:
vetcompass_df = pd.read_csv("../Data/vet_compass/STR024A03 20190719 sample x10,000.csv", header=None, names = columns)

In [ ]:
vetcompass_df.head()

In [ ]:
len(vetcompass_df['Patient Number'].unique())

In [ ]:
len(vetcompass_df)

# Examination Text Cleaning

In [ ]:
# Start with Patient Number and Examination Text cols only
examination_text = vetcompass_df[['Consultation Number', 'Consultation Date', 'Patient Number', 'Examination Text']]

In [ ]:
# Drop rows with NaN as the examination text
examination_text_clean = examination_text.copy()
examination_text_clean = examination_text_clean.dropna(subset=['Examination Text'])

print(len(examination_text_clean))
print(len(examination_text_clean['Patient Number'].unique()))
print(len(examination_text_clean['Consultation Date'].unique()))
print(len(examination_text_clean['Consultation Number'].unique()))
print(len(examination_text_clean['Examination Text'].unique()))

In [ ]:
# Lower case in examination text
examination_text_clean['Examination Text'] = examination_text_clean['Examination Text'].str.lower()

In [ ]:
examination_text_clean['ExaminationTextLen'] = examination_text_clean['Examination Text'].str.len()

In [ ]:
#examination_text_clean.sort_values(by=['ExaminationTextLen'], ascending=False)
print(examination_text_clean['ExaminationTextLen'].min())
print(examination_text_clean['ExaminationTextLen'].max())

In [ ]:
examination_text_clean['ExaminationTextLen'].hist(bins=100, range=[0, 100])
plt.show()

In [ ]:
longform = examination_text_clean.loc[examination_text_clean['ExaminationTextLen'] == 22000]
longform.iloc[0]

## Investigate dropping duplicate rows for examination text

Check duplicates accross all columns, as there may be cases where simple examination text gets repeated, but is for a different patient/consult date/consult number

Might need to exclude consultation date, as this can vary by minutes

In [ ]:
examination_text_dropduplicates = examination_text_clean[['Consultation Number', 'Patient Number', 'Examination Text']].drop_duplicates(keep='first')

In [ ]:
examination_text_dropduplicates.head(n=10)

In [ ]:
len(examination_text_dropduplicates['Consultation Number'].unique())

In [ ]:
len(examination_text_dropduplicates)

## Pull out different string patterns

In [ ]:
len(examination_text_dropduplicates.loc[examination_text_dropduplicates['Examination Text'].str.contains('blood')])

In [ ]:
examination_text_blood = examination_text_dropduplicates.loc[examination_text_dropduplicates['Examination Text'].str.contains('blood')]

for i in range(20):
    print(examination_text_blood.iloc[i]['Examination Text'])

In [ ]:
examination_text_vetnostics = examination_text_dropduplicates.loc[examination_text_dropduplicates['Examination Text'].str.contains('vetnostics')]

print(len(examination_text_vetnostics['Examination Text']))

for i in range(3):
    print(examination_text_vetnostics.iloc[i]['Examination Text'])

# up to 2 words followed by a colon

In [ ]:
def get_patterns(df, pattern):
    note_captions_lists = [re.findall(string=text, pattern=pattern) for text in df['Examination Text'] if isinstance(text, str)]
    note_captions = [text for text_list in note_captions_lists for text in text_list]
    note_captions_set = set(note_captions)
    return note_captions_set

In [ ]:
get_patterns(vetcompass_df, r"(?:[A-Z][a-z]+ )?[A-Z][a-z]+\:")

In [ ]:
len(list(get_patterns(vetcompass_df, r"(?:[A-Z][a-z]+ )?[A-Z][a-z]+\:")))

# 1 word followed by a colon

In [ ]:
len(list(get_patterns(vetcompass_df, r"[A-Z][a-z]+\:")))

In [ ]:
get_patterns(vetcompass_df, r"[A-Z][a-z]+\:")

In [ ]:
#sorted(get_patterns(vetcompass_df, r"[A-Z][a-z]+\:"))

In [ ]:
vetcompass_df_wellbehaved = vetcompass_df[vetcompass_df['Examination Text'].apply(lambda text: bool(re.search(string=text, pattern='Appointment Notes\:')) if isinstance(text, str) else False)]

In [ ]:
vetcompass_df_wellbehaved['Examination Text']

In [ ]:
vetcompass_df_wellbehaved.shape

In [ ]:
vetcompass_df.shape

In [ ]:
print('about 1/3 are well behaved- they have Appointment Notes: string in Examination Text')

In [ ]:
vetcompass_df[vetcompass_df['Examination Text'].notnull()].shape

In [ ]:
single_word_patterns = get_patterns(vetcompass_df_wellbehaved, r"[A-Z][a-z]+\:")

In [ ]:
single_word_patterns

In [ ]:
vetcompass_df_wellbehaved['Examination Text'][vetcompass_df_wellbehaved['Examination Text'].str.contains('Jazzy:')].iloc[0]

In [ ]:
vetcompass_df_wellbehaved['Examination Text'].iloc[5]

In [ ]:
single_word_patterns_with_count = []
for word_pattern in single_word_patterns:
    single_word_patterns_with_count.append((word_pattern, vetcompass_df_wellbehaved['Examination Text'].str.contains(word_pattern).sum()))

In [ ]:
sorted(single_word_patterns_with_count, key=lambda pair: pair[1], reverse=True)

# look at examination text which contain 'Laboratory' in well behaved/other

In [ ]:
get_patterns(vetcompass_df_wellbehaved, r"Laboratory:")

In [ ]:
# df which has well behaved with 'Laboratory:'
# :/ slash is escape key which means use colon as colon
wellbehaved_Laboratory = vetcompass_df_wellbehaved[vetcompass_df_wellbehaved['Examination Text'].apply(lambda text: bool(re.search(string=text, pattern='Laboratory\:')) if isinstance(text, str) else False)]

In [ ]:
len(wellbehaved_Laboratory)

In [ ]:
wellbehaved_Laboratory['Examination Text']

In [ ]:
# how do we see text after 'Laboratory:'?

In [ ]:
get_patterns(wellbehaved_Laboratory, "Laboratory\:.{1,100}")

In [ ]:
get_patterns(wellbehaved_Laboratory, "Bloods\:.{1,150}")

In [ ]:
get_patterns(wellbehaved_Laboratory, "Assessment\:.{1,150}")

In [ ]:
# look at all data

In [ ]:
len(get_patterns(vetcompass_df, "cre|CRE.{1,150}"))

In [ ]:
get_patterns(vetcompass_df, "cre|CRE.{1,150}")

In [ ]:
get_patterns(vetcompass_df, "CKD|ckd|Kidney|kidney|renal.{1,150}")

In [ ]:
len(get_patterns(vetcompass_df, "CKD|ckd|Kidney|kidney|renal"))

In [ ]:
len(get_patterns(vetcompass_df, "CKD|ckd|Kidney|kidney|renal.{1,150}"))

In [ ]:
len(get_patterns(vetcompass_df, "CKD.{1,150}|ckd.{1,150}|Kidney.{1,150}|kidney.{1,150}|renal.{1,150}|RENAL.{1,150}"))

In [ ]:
# kidney related records

In [ ]:
kidney_related_df = vetcompass_df[vetcompass_df['Examination Text'].apply(lambda text: bool(re.search(string=text, pattern='CKD|ckd|Kidney|kidney|renal')) if isinstance(text, str) else False)]

In [ ]:
len(kidney_related_df)

In [ ]:
kidney_related_patients = kidney_related_df['Patient Number'].unique()

In [ ]:
len(kidney_related_df['Patient Number'].unique())

In [ ]:
list(kidney_related_patients)

In [ ]:
chronic_kidney_related_df = kidney_related_df[kidney_related_df['Examination Text'].apply(lambda text: bool(re.search(string=text, pattern='chronic|CHRONIC|Chronic|CKD|ckd|Ckd')) if isinstance(text, str) else False)]

In [ ]:
len(chronic_kidney_related_df)

In [ ]:
len(chronic_kidney_related_df['Patient Number'].unique())

In [ ]:
chronic_kidney_related_df.iloc[0]['Examination Text']

# try looking for k/d
Look in Examination Text, Item Name, Item Label (hardly any here)

In [ ]:
find_kd = vetcompass_df[vetcompass_df['Item Name'].apply(lambda text: bool(re.search(string=text, pattern='k/d|K/D|K/d')) if isinstance(text, str) else False)]

In [ ]:
len(find_kd)

In [ ]:
len(find_kd['Patient Number'].unique())

In [ ]:
find_kd.iloc[100]['Item Name']

In [ ]:
chronic_related_df['Examination Text']

In [ ]:
chronic_related_df = vetcompass_df[vetcompass_df['Examination Text'].apply(lambda text: bool(re.search(string=text, pattern='chronic|CHRONIC|Chronic')) if isinstance(text, str) else False)]

In [ ]:
len(chronic_related_df)

In [ ]:
get_patterns(vetcompass_df, "chronic.{1,150}")

In [ ]:
get_patterns(vetcompass_df, "semintra.{1,150}")

In [ ]:
vetcompass_df[vetcompass_df['Examination Text'].apply(lambda text: bool(re.search(string=text, pattern='semintra')) if isinstance(text, str) else False)]

In [ ]:
vetcompass_df[vetcompass_df['Item Name'].apply(lambda text: bool(re.search(string=text, pattern='Semintra')) if isinstance(text, str) else False)]

In [ ]:
vetcompass_df[vetcompass_df['Item Label'].apply(lambda text: bool(re.search(string=text, pattern='Semintra')) if isinstance(text, str) else False)]

# How do we pull out individual blood values?